In [1]:
# generated by Claude https://claude.ai/chat/979f9800-5653-46be-a8e2-c33fb149594b

In [2]:
import h3
import geopandas as gpd
import rasterio
import numpy as np
import pandas as pd
from shapely.geometry import Polygon, MultiPolygon
import pyproj

In [3]:
# Load the raster data
with rasterio.open('../data/Day 1.tif') as src:
    raster_data = src.read(1)  # Assuming single band
    transform = src.transform

In [4]:
# Load the ZIP code data
zip_gdf = gpd.read_file('../data/cb_2020_us_zcta520_500k.shp')

# Print information about the GeoDataFrame
print("GeoDataFrame Info:")
print(zip_gdf.info())
print("\nGeometry Types:")
print(zip_gdf.geometry.type.value_counts())
print("\nBounds:")
print(zip_gdf.total_bounds)

# Check if CRS is set
if zip_gdf.crs is None:
    print("\nWARNING: CRS is not set. Setting to EPSG:4326 (WGS84).")
    zip_gdf = zip_gdf.set_crs(epsg=4326, allow_override=True)
else:
    print(f"\nCurrent CRS: {zip_gdf.crs}")

# Ensure CRS is set to a known projected CRS
target_crs = 'EPSG:3857'  # Web Mercator
zip_gdf = zip_gdf.to_crs(target_crs)
print(f"Projected CRS: {zip_gdf.crs}")


GeoDataFrame Info:
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 33791 entries, 0 to 33790
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   ZCTA5CE20   33791 non-null  object  
 1   AFFGEOID20  33791 non-null  object  
 2   GEOID20     33791 non-null  object  
 3   NAME20      33791 non-null  object  
 4   LSAD20      33791 non-null  object  
 5   ALAND20     33791 non-null  int64   
 6   AWATER20    33791 non-null  int64   
 7   geometry    33791 non-null  geometry
dtypes: geometry(1), int64(2), object(5)
memory usage: 2.1+ MB
None

Geometry Types:
Polygon         32139
MultiPolygon     1652
Name: count, dtype: int64

Bounds:
[-176.696676  -14.37374   145.830418   71.341223]

Current CRS: EPSG:4269
Projected CRS: EPSG:3857


In [5]:
# Function to convert raster cell to polygon
def raster_cell_to_polygon(row, col, transform):
    corners = [
        transform * (col, row),
        transform * (col+1, row),
        transform * (col+1, row+1),
        transform * (col, row+1)
    ]
    return Polygon(corners)

# Function to find optimal H3 resolution
def find_optimal_resolution(zip_gdf, raster_cell_area):
    avg_zip_area = zip_gdf.geometry.area.mean()
    for res in range(5, 11):  # Test resolutions 5 to 10
        hex_area = h3.hex_area(res, unit='m^2')
        ratio = avg_zip_area / hex_area
        if 10 < ratio < 1000:  # Adjust these thresholds as needed
            return res
    return 8  # Default to resolution 8 if no optimal found

In [6]:
# Calculate raster cell area
cell_height = transform[0]
cell_width = abs(transform[4])
raster_cell_area = cell_height * cell_width

In [7]:
# Find optimal H3 resolution
optimal_res = find_optimal_resolution(zip_gdf, raster_cell_area)
print(f"Optimal H3 resolution: {optimal_res}")

Optimal H3 resolution: 6


In [8]:
def polygon_to_h3(geometry, resolution):
    try:
        # Check if the geometry is valid
        if not geometry.is_valid:
            print(f"Invalid geometry found: {geometry}")
            return []
        
        # Convert geometry to geographic coordinates for H3
        geo_geometry = gpd.GeoSeries([geometry], crs=zip_gdf.crs).to_crs(4326).iloc[0]
        
        if isinstance(geo_geometry, Polygon):
            return list(h3.polyfill(geo_geometry.__geo_interface__, resolution))
        elif isinstance(geo_geometry, MultiPolygon):
            hexagons = []
            for polygon in geo_geometry.geoms:
                hexagons.extend(h3.polyfill(polygon.__geo_interface__, resolution))
            return list(set(hexagons))  # Remove duplicates
        else:
            print(f"Unsupported geometry type: {type(geo_geometry)}")
            return []
    except Exception as e:
        print(f"Error processing geometry: {e}")
        return []

In [9]:
# Apply the function to the ZIP code geometries
zip_gdf['h3_hexagons'] = zip_gdf.geometry.apply(lambda x: polygon_to_h3(x, optimal_res))

In [10]:
# Create a dictionary of H3 index to ZIP code
h3_to_zip = {h: zip_code for zip_code, hexagons in zip(zip_gdf.index, zip_gdf.h3_hexagons) for h in hexagons}

In [ ]:
# Process raster data
rows, cols = raster_data.shape
h3_values = {}

for row in range(rows):
    for col in range(cols):
        if raster_data[row, col] != src.nodata:
            cell_poly = raster_cell_to_polygon(row, col, transform)
            cell_h3 = h3.polyfill(cell_poly.__geo_interface__, optimal_res)
            for h in cell_h3:
                if h in h3_to_zip:
                    if h not in h3_values:
                        h3_values[h] = []
                    h3_values[h].append(raster_data[row, col])

In [ ]:
# Calculate average value for each H3 hexagon
h3_avg_values = {h: np.mean(values) for h, values in h3_values.items()}

In [ ]:
# Aggregate back to ZIP codes
zip_gdf['weighted_value'] = zip_gdf.apply(lambda row: np.mean([h3_avg_values.get(h, 0) for h in row.h3_hexagons]), axis=1)

In [ ]:
# Display results
print(zip_gdf[['weighted_value']])

In [ ]:
zip_gdf.plot()